In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup, SoupStrainer
from textual_analysis.tools_scrapping import gather_links

In [2]:
def clean_links(links):
    clean_links_array = []
    for link in links:
        if "quote" in link and "-USD" in link:
            clean_links_array.append(link.replace("/quote/",""))
            
    return clean_links_array

In [3]:
symbols = []

for offset in np.arange(0,9900,100):
    url = f"https://finance.yahoo.com/screener/predefined/all_cryptocurrencies_us/?count=100&offset={offset}"

    resp = requests.get(
        url,
        headers={
            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36"
        },
    )
    
    links = gather_links(resp)
    
    clean_links_array = clean_links(links)
    
    symbols = symbols + clean_links_array

/home/ignacio/Documents/SFU/crypto/textual_analysis/tools_scrapping.py:19: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 19 of the file /home/ignacio/Documents/SFU/crypto/textual_analysis/tools_scrapping.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  for link in BeautifulSoup(response.content, parse_only=SoupStrainer('a')):


In [4]:
stablecoins = list(pd.read_csv('stablecoins.csv').keys())
stablecoins = list(map(lambda x: x.lower(), stablecoins))

In [5]:
symbol_non_stablecoins = []
symbol_stablecoins     = []

for symbol in symbols:
    
    cmp = symbol.replace("-USD", "").lower()
    if cmp in stablecoins:
        symbol_stablecoins.append(symbol)
    else:
        symbol_non_stablecoins.append(symbol)

In [6]:
print(len(symbol_non_stablecoins))
print(len(symbol_stablecoins))

9811
86


In [ ]:
i = 0
for symbol in symbols:
    msft = yf.Ticker(symbol)
    temp_data = msft.history(period="max")
    
    nrows = temp_data.shape[0]
    ticker = [symbol] * nrows
    temp_data = temp_data.drop(columns = ['Dividends', "Stock Splits"])
    temp_data['close_times_volume'] = temp_data['Close'] * temp_data['Volume']
    temp_data['Ticker'] = ticker
    
    if symbol in symbol_stablecoins:
        temp_data['stablecoin'] = [True]  * nrows
    else:
        temp_data['stablecoin'] = [False] * nrows
        
    if i == 0:
        data = temp_data.copy()
    else:
        data = pd.concat([data, temp_data],axis = 0)
    
    i+=1
    
compression_opts = dict(method='zip',
                        archive_name='data_crypto_25_april.csv')
data.to_csv('data_crypto_25_april.zip', index=False,
          compression=compression_opts) 

In [ ]:
list(data[data.stablecoin == True]['Ticker'].unique())